# Indonesian ULMFiT from scratch

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai import *
from fastai.text import *

In [2]:
# bs=48
 bs=24
# bs=128

In [3]:
torch.cuda.set_device(0)

In [4]:
data_path = Config.data_path()
print(data_path)

/home/arvinfebriyan/.fastai/data


This will create a `idwiki` folder, containing a `idwiki` text file with the wikipedia contents. (For other languages, replace `id` with the appropriate code from the [list of wikipedias](https://meta.wikimedia.org/wiki/List_of_Wikipedias).)

In [5]:
lang = 'id'
# lang = 'zh'

In [6]:
name = f'{lang}wiki'
path = data_path/name
path.mkdir(exist_ok=True, parents=True)
lm_fns = [f'{lang}_wt', f'{lang}_wt_vocab']

In [7]:
print(path)

/home/arvinfebriyan/.fastai/data/idwiki


## Indonesian wikipedia model

### Download data

In [8]:
from fast_nlputils import split_wiki,get_wiki

In [9]:
get_wiki(path,lang)

/home/arvinfebriyan/.fastai/data/idwiki/idwiki already exists; not downloading


In [10]:
path.ls()

[PosixPath('/home/arvinfebriyan/.fastai/data/idwiki/idwiki-latest-pages-articles.xml'),
 PosixPath('/home/arvinfebriyan/.fastai/data/idwiki/log'),
 PosixPath('/home/arvinfebriyan/.fastai/data/idwiki/wikiextractor'),
 PosixPath('/home/arvinfebriyan/.fastai/data/idwiki/idwiki-latest-pages-articles.xml.bz2'),
 PosixPath('/home/arvinfebriyan/.fastai/data/idwiki/idwiki'),
 PosixPath('/home/arvinfebriyan/.fastai/data/idwiki/docs')]

In [11]:
!head -n4 {path}/{name}

<doc id="1" url="https://id.wikipedia.org/wiki?curid=1" title="Asam deoksiribonukleat">
Asam deoksiribonukleat

Asam deoksiribonukleat, lebih dikenal dengan singkatan DNA (bahasa Inggris: "deoxyribonucleic acid"), adalah sejenis biomolekul yang menyimpan dan menyandi instruksi-instruksi genetika setiap organisme dan banyak jenis virus. Instruksi-instruksi genetika ini berperan penting dalam pertumbuhan, perkembangan, dan fungsi organisme dan virus. DNA merupakan asam nukleat; bersamaan dengan protein dan karbohidrat, asam nukleat adalah makromolekul esensial bagi seluruh makhluk hidup yang diketahui. Kebanyakan molekul DNA terdiri dari dua unting biopolimer yang berpilin satu sama lainnya membentuk heliks ganda. Dua unting DNA ini dikenal sebagai polinukleotida karena keduanya terdiri dari satuan-satuan molekul yang disebut nukleotida. Tiap-tiap nukleotida terdiri atas salah satu jenis basa nitrogen (guanina (G), adenina (A), timina (T), atau sitosina (C)), gula monosakarida yang diseb

This function splits the single wikipedia file into a separate file per article. This is often easier to work with.

In [12]:
dest = split_wiki(path,lang)

/home/arvinfebriyan/.fastai/data/idwiki/docs already exists; not splitting


In [13]:
dest.ls()[:5]

[PosixPath('/home/arvinfebriyan/.fastai/data/idwiki/docs/Indra Thohir.txt'),
 PosixPath('/home/arvinfebriyan/.fastai/data/idwiki/docs/Koalisi Jamaika.txt'),
 PosixPath('/home/arvinfebriyan/.fastai/data/idwiki/docs/Etnografi.txt'),
 PosixPath('/home/arvinfebriyan/.fastai/data/idwiki/docs/Married with Children.txt'),
 PosixPath('/home/arvinfebriyan/.fastai/data/idwiki/docs/Pantai Widuri.txt')]

In [14]:
# Use this to convert Chinese traditional to simplified characters
# ls *.txt | parallel -I% opencc -i % -o ../zhsdocs/% -c t2s.json

### Create pretrained model

In [15]:
#data = (TextList.from_folder(dest)
#            .split_by_rand_pct(0.1, seed=42)
#            .label_for_lm()           
#            .databunch(bs=bs, num_workers=1))

In [16]:
#data.save(f'{lang}_databunch')

In [17]:
#len(data.vocab.itos),len(data.train_ds)

In [18]:
path_databunch = path/'docs'
print(path_databunch)

/home/arvinfebriyan/.fastai/data/idwiki/docs


In [19]:
bs = 16
data = load_data(dest, f'{lang}_databunch', bs=bs)

In [20]:
torch.cuda.empty_cache()

In [21]:
learn = language_model_learner(data, AWD_LSTM, drop_mult=0.5, pretrained=False).to_fp16()

In [22]:
lr = 1e-2
lr *= bs/48  # Scale learning rate by batch size

In [23]:
learn.unfreeze()
learn.fit_one_cycle(2, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,4.410362,4.282310,0.324866,5:57:26
1,4.255765,4.002489,0.352433,5:47:19


Save the pretrained model and vocab:

In [24]:
mdl_path = path/'models'
mdl_path.mkdir(exist_ok=True)
learn.to_fp32().save(mdl_path/lm_fns[0], with_opt=False)
learn.data.vocab.save(mdl_path/(lm_fns[1] + '.pkl'))

In [30]:
TEXT = "Kamu ini"
N_WORDS = 40
N_SENTENCES = 10

print("\n".join(learn.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))

Kamu ini akan memperoleh sekadar " Sri Lanka " di mana ia bukanlah seorang Raja , namun ia tidak pernah ingin berperang dengannya . 
 
  Dalam posisi yang lebih rendah , ketika Raja Dewan Agung
Kamu ini adalah salah satu dari beberapa tokoh penting di Indonesia . 
 
  Ada beberapa organisasi yang memiliki tugas lainnya yakni : 
 
  Non - Anggota Dewan Pimpinan Pusat : 
 
  Kelompok Kegiatan
Kamu ini memiliki beberapa peristiwa penting yang ada di dalam PT Angkasa Pura , seperti " Public Angkatan Udara Angkatan Udara 3 " , " Pasukan Militer Jepang Indonesia "
Kamu ini , Sang Pencipta Tanah , akan Asisten ! 
  Kami 
  akan menjadi dan diarahkan ke jawab Tuhan dari Tuhan . 
  Kapan Aku telah mengepalai Bumi ,
Kamu ini berjalan jauh dari jalan . Aku tidak bisa melihat apa yang akan terjadi dalam novel itu . Jika ia tidak bermaksud untuk mengambil referensi , dia akan menjadi anak yang paling liar . Baru - baru ini
Kamu ini hak - hak warga di Indonesia . Dalam konteks ini , Ia melakukan seg

## Indonesian sentiment analysis - to be continued

### Language model

- [Data](https://github.com/ngxbac/aivivn_phanloaisacthaibinhluan/tree/master/data)
- [Competition details](https://www.aivivn.com/contests/1)
- Top 3 f1 scores: 0.900, 0.897, 0.897

In [ ]:
train_df = pd.read_csv(path/'train.csv')
train_df.loc[pd.isna(train_df.comment),'comment']='NA'
train_df.head()

In [ ]:
test_df = pd.read_csv(path/'test.csv')
test_df.loc[pd.isna(test_df.comment),'comment']='NA'
test_df.head()

In [ ]:
df = pd.concat([train_df,test_df], sort=False)

In [ ]:
data_lm = (TextList.from_df(df, path, cols='comment')
    .split_by_rand_pct(0.1, seed=42)
    .label_for_lm()           
    .databunch(bs=bs, num_workers=1))

In [ ]:
learn_lm = language_model_learner(data_lm, AWD_LSTM, pretrained_fnames=lm_fns, drop_mult=1.0)

In [ ]:
lr = 1e-3
lr *= bs/48

In [ ]:
learn_lm.fit_one_cycle(2, lr*10, moms=(0.8,0.7))

In [ ]:
learn_lm.unfreeze()
learn_lm.fit_one_cycle(8, lr, moms=(0.8,0.7))

In [ ]:
learn_lm.save(f'{lang}fine_tuned')
learn_lm.save_encoder(f'{lang}fine_tuned_enc')

### Classifier

In [ ]:
data_clas = (TextList.from_df(train_df, path, vocab=data_lm.vocab, cols='comment')
    .split_by_rand_pct(0.1, seed=42)
    .label_from_df(cols='label')
    .databunch(bs=bs, num_workers=1))

data_clas.save(f'{lang}_textlist_class')

In [ ]:
data_clas = load_data(path, f'{lang}_textlist_class', bs=bs, num_workers=1)

In [ ]:
from sklearn.metrics import f1_score

@np_func
def f1(inp,targ): return f1_score(targ, np.argmax(inp, axis=-1))

In [ ]:
learn_c = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5, metrics=[accuracy,f1]).to_fp16()
learn_c.load_encoder(f'{lang}fine_tuned_enc')
learn_c.freeze()

In [ ]:
lr=2e-2
lr *= bs/48

In [ ]:
learn_c.fit_one_cycle(2, lr, moms=(0.8,0.7))

In [ ]:
learn_c.fit_one_cycle(2, lr, moms=(0.8,0.7))

In [ ]:
learn_c.freeze_to(-2)
learn_c.fit_one_cycle(2, slice(lr/(2.6**4),lr), moms=(0.8,0.7))

In [ ]:
learn_c.freeze_to(-3)
learn_c.fit_one_cycle(2, slice(lr/2/(2.6**4),lr/2), moms=(0.8,0.7))

In [ ]:
learn_c.unfreeze()
learn_c.fit_one_cycle(1, slice(lr/10/(2.6**4),lr/10), moms=(0.8,0.7))

In [ ]:
learn_c.save(f'{lang}clas')

Competition top 3 f1 scores: 0.90, 0.89, 0.89. Winner used an ensemble of 4 models: TextCNN, VDCNN, HARNN, and SARNN.

## Ensemble

In [ ]:
data_clas = load_data(path, f'{lang}_textlist_class', bs=bs, num_workers=1)
learn_c = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5, metrics=[accuracy,f1]).to_fp16()
learn_c.load(f'{lang}clas', purge=False);

In [ ]:
preds,targs = learn_c.get_preds(ordered=True)
accuracy(preds,targs),f1(preds,targs)

In [ ]:
data_clas_bwd = load_data(path, f'{lang}_textlist_class_bwd', bs=bs, num_workers=1, backwards=True)
learn_c_bwd = text_classifier_learner(data_clas_bwd, AWD_LSTM, drop_mult=0.5, metrics=[accuracy,f1]).to_fp16()
learn_c_bwd.load(f'{lang}clas_bwd', purge=False);

In [ ]:
preds_b,targs_b = learn_c_bwd.get_preds(ordered=True)
accuracy(preds_b,targs_b),f1(preds_b,targs_b)

In [ ]:
preds_avg = (preds+preds_b)/2

In [ ]:
accuracy(preds_avg,targs_b),f1(preds_avg,targs_b)

## Resources

- https://github.com/fastai/course-nlp